In [8]:
import plotly.offline as pyo
import plotly_express as pyx
import plotly.graph_objects as go
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
import plotly.figure_factory as ff
import numpy as np
from numpy import *
import pandas as pd

data = pd.read_csv("sample_semanai.csv")
data = data.drop_duplicates().iloc[:]

In [9]:
data2 = data[data["Acción"] != "Cancelación"]
data2.dropna(axis=0, subset=['Clave platillo'], inplace=True)

In [10]:
data2 = data2[['Orden','Cantidad','Clave platillo', 'idEncuesta']]

In [11]:
data2 = (data2.groupby(['idEncuesta', 'Clave platillo'])['Cantidad'].sum().unstack().reset_index())
transform_NaNs = isnan(data2)
data2[transform_NaNs] = 0
data2 = data2.dropna()
data2

Clave platillo,idEncuesta,BAS1,BAS10,BAS11,BAS12,BAS13,BAS14,BAS2,BAS3,BAS4,...,bebi004,bebi005,bebi006,bebi01,bebi02,bebi03,mcl90004,mcl90005,mcl90009,mcla9006
0,115787,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,115788,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,115789,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,115791,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,115793,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16134,615470,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16135,615569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16136,615571,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16137,615573,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
top_5 = data.groupby(['Clave platillo'])['Cantidad'].agg(
    {"Cuenta": len}).sort_values(
    "Cuenta", ascending=False).head(5).reset_index()

/home/kaetahr/.local/lib64/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning:

using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)




In [13]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

data2 = data2.applymap(encode_units)
data2.drop('idEncuesta',inplace=True, axis=1)
itemsets = apriori(data2, min_support = 0.015, use_colnames=True)
rules = association_rules(itemsets, metric="lift", min_threshold=1)
rules = pd.DataFrame(rules)
rules = rules.drop(['antecedent support', 'consequent support','leverage','conviction'],axis=1)
rules

,antecedents,consequents,support,confidence,lift
0,(BAS1),(HAM12),0.019580,0.135274,1.111602
1,(HAM12),(BAS1),0.019580,0.160896,1.111602
2,(BAS1),(MHAM30),0.048516,0.335188,1.035926
3,(MHAM30),(BAS1),0.048516,0.149943,1.035926
4,(MHAM30),(BAS50),0.027449,0.084833,1.067961
5,(BAS50),(MHAM30),0.027449,0.345554,1.067961
6,(BCA8),(MBCA38),0.016606,0.170267,4.737821
7,(MBCA38),(BCA8),0.016606,0.462069,4.737821
8,(HAM1),(MHAM30),0.030113,0.397383,1.228145
9,(MHAM30),(HAM1),0.030113,0.093068,1.228145


In [14]:
#Index = top_5['Clave platillo']
#rules = pd.DataFrame.to_string(rules, columns=['antecedents','consequents'])
#rules = pd.to_numeric(rules, columns=['support','confidence','lift'])
rules = rules.applymap(pd.to_numeric)
dendo = ff.create_dendrogram(rules, orientation="right", lab)
dendo.show()
rules.dtypes

SyntaxError: positional argument follows keyword argument (<ipython-input-14-412cb5ccbceb>, line 5)